In [4]:
# import libraries
import os
import requests
import pandas as pd
import numpy as np
from pandas import json_normalize 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# for regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# for metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score, roc_auc_score 

# for plots 
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from seaborn import pairplot

# model save/load
import pickle

In [5]:
# get api key
api_key = os.environ['SPORTS_DATA_IO_API']
print(api_key)

4baa2aa7dbd644bc926c3818dde19936


In [278]:
def custom_regular_season_dataframe(season):
    # get custom dataframes
    if season != 2020:
        weeks = 17
    else:
        weeks = 11
        
    # load current season data    
    df_current_season = pd.read_csv('./data/season_team_data/{0}_data.csv'.format(str(season)))
    
    
    # bool for loading in past season
    previous_season_exists = False
    # load in the past seasons data if it exists
    try:
        df_past_season = pd.read_csv('./data/season_team_data/{0}_data.csv'.format(str(season - 1)))
        df_past_season = df_past_season.set_index('Team')
    except:
        previous_season_exists=False
    else:
        previous_season_exists=True

    
        
    # create new data frame without week 1 columns
    columns=['TotalScore','TotalWins', 'TotalFirstDowns', 'ThirdDownPercentage', 'TotalTimeOfPossession', 'GamesPlayed']
    current_season_dataframe =  pd.DataFrame(index=df_current_season['Team'], columns=columns)
    current_season_dataframe = current_season_dataframe.fillna(0)

    # populate week 1 averages with data from previous season
    if previous_season_exists:
        for team, row in current_season_dataframe.iterrows():
            current_season_dataframe.at[team, 'avg_up_to_week_1'] = df_past_season.loc[team,'Score'] / 16
            current_season_dataframe.at[team, 'first_downs_up_to_week_1'] = df_past_season.loc[team,'FirstDowns'] / 16
            current_season_dataframe.at[team, 'wins_up_to_week_1'] = 0.0
            current_season_dataframe.at[team, 'third_down_percentage_up_to_week_1'] = df_past_season.loc[team,'ThirdDownPercentage'] 
            
            # convert string time of possession to float
            top = df_past_season.loc[team,'TimeOfPossession'].split(':')
            top = float('{0}.{1}'.format(top[0],top[1]))
            current_season_dataframe.at[team, 'time_of_possession_up_to_week_1'] = top
    
    # loop through each regular seasons weeks games
    for week in range(1, weeks + 1):
        df_week_data = pd.read_csv('./data/season_game_data/{0}/week_{1}_data.csv'.format(season, week))
        for _, row in df_week_data.iterrows():
            # ---------------------------------------update each teams score per week------------------------------------------
            current_season_dataframe.loc[row.Team,row.Week] = row.Score
            current_season_dataframe[row.Week] = current_season_dataframe[row.Week].fillna(0)
            
            # ---------------------------------------update each teams games played -------------------------------------------
            current_season_dataframe.loc[row.Team,'GamesPlayed'] += 1
            
            # ----------------------------------------------update score to week------------------------------------------------
            current_season_dataframe.loc[row.Team,'TotalScore'] += row.Score 
            
            update_avg_score = current_season_dataframe.loc[row.Team,'TotalScore'] / current_season_dataframe.loc[row.Team,'GamesPlayed']
            
            current_season_dataframe.loc[row.Team,'avg_up_to_week_{0}'.format(row.Week + 1)] = update_avg_score 
            
            current_season_dataframe['avg_up_to_week_{0}'.format(row.Week + 1)] = current_season_dataframe['avg_up_to_week_{0}'.format(row.Week + 1)].fillna(current_season_dataframe['TotalScore'] / current_season_dataframe.loc[row.Team,'GamesPlayed'])

            # ----------------------------------- store wins per team up to a certain week ----------------------------------
            if row.Score > row.OpponentScore:
                current_season_dataframe.loc[row.Team,'TotalWins'] += 1
            
            current_season_dataframe.loc[row.Team,'wins_up_to_week_{0}'.format(row.Week + 1)] = current_season_dataframe.loc[row.Team,'TotalWins']
            
            current_season_dataframe['wins_up_to_week_{0}'.format(row.Week + 1)] = current_season_dataframe['wins_up_to_week_{0}'.format(row.Week + 1)].fillna(0)
    
            # ----------------------------------- store first downs per team up to certain week ----------------------------------
            current_season_dataframe.loc[row.Team,'TotalFirstDowns'] += row.FirstDowns 
                    
            update_first_downs = current_season_dataframe.loc[row.Team,'TotalFirstDowns'] / current_season_dataframe.loc[row.Team,'GamesPlayed']
            current_season_dataframe.loc[row.Team,'first_downs_up_to_week_{0}'.format(row.Week + 1)] = update_first_downs
            
            current_season_dataframe['first_downs_up_to_week_{0}'.format(row.Week + 1)] = current_season_dataframe['first_downs_up_to_week_{0}'.format(row.Week + 1)].fillna(current_season_dataframe['TotalFirstDowns'] / current_season_dataframe.loc[row.Team,'GamesPlayed'])

            # ----------------------------------- calculate third down % per team up to certain week ----------------------------------
            current_season_dataframe.loc[row.Team,'ThirdDownPercentage'] += row.ThirdDownPercentage 
    
            update_third_down_percentage = current_season_dataframe.loc[row.Team,'ThirdDownPercentage'] / current_season_dataframe.loc[row.Team,'GamesPlayed']

            current_season_dataframe.loc[row.Team,'third_down_percentage_up_to_week_{0}'.format(row.Week + 1)] = update_third_down_percentage

        
            current_season_dataframe['third_down_percentage_up_to_week_{0}'.format(row.Week + 1)] = current_season_dataframe['third_down_percentage_up_to_week_{0}'.format(row.Week + 1)].fillna(current_season_dataframe['ThirdDownPercentage'] / current_season_dataframe.loc[row.Team,'GamesPlayed'])

            # ----------------------------------- calculate average time of possession per team ----------------------------------
            team_top = float('{0}.{1}'.format(row.TimeOfPossessionMinutes,row.TimeOfPossessionSeconds))
            
            current_season_dataframe.loc[row.Team,'TotalTimeOfPossession'] += team_top
            
            update_time_of_possession = current_season_dataframe.loc[row.Team,'TotalTimeOfPossession'] / current_season_dataframe.loc[row.Team,'GamesPlayed']
            
            current_season_dataframe.loc[row.Team,'time_of_possession_up_to_week_{0}'.format(row.Week + 1)] = update_time_of_possession

            current_season_dataframe['time_of_possession_up_to_week_{0}'.format(row.Week + 1)] = current_season_dataframe['time_of_possession_up_to_week_{0}'.format(row.Week + 1)].fillna(current_season_dataframe['TotalTimeOfPossession'] / current_season_dataframe.loc[row.Team,'GamesPlayed'])
        

    return current_season_dataframe
    
    

In [82]:
def custom_post_season_dataframe(season):
    
    # load that seasons end of season data
    df_current_season = pd.read_csv('./data/custom_season_team_data/{0}_data.csv'.format(str(season)))
    
    
    columns=['TotalScore', 'TotalFirstDowns', 'ThirdDownPercentage', 'TotalTimeOfPossession', 'GamesPlayed']
    current_season_dataframe =  pd.DataFrame(index=df_current_season['Team'], columns=columns)
    current_season_dataframe = current_season_dataframe.fillna(0)

    
    # set index to be team name
    df_current_season = df_current_season.set_index('Team')
    
    # populate week 1 averages with data from previous season
    for team, row in current_season_dataframe.iterrows():
        current_season_dataframe.at[team, 'avg_up_to_week_1_playoffs'] = df_current_season.loc[team,'avg_up_to_week_18']
        current_season_dataframe.at[team, 'first_downs_up_to_week_1_playoffs'] = df_current_season.loc[team,'first_downs_up_to_week_18']
        current_season_dataframe.at[team, 'wins_up_to_week_1_playoffs'] = df_current_season.loc[team,'wins_up_to_week_18'] 
        current_season_dataframe.at[team, 'third_down_percentage_up_to_week_1_playoffs'] = df_current_season.loc[team,'third_down_percentage_up_to_week_18'] 
        current_season_dataframe.at[team, 'time_of_possession_up_to_week_1_playoffs'] = df_current_season.loc[team,'time_of_possession_up_to_week_18']
        
        '''
        setting this specifically for bye teams, as the values they have will be NaN,
        we need their week 2 value to be the same as week 1
        all other teams that play a game will have it overwritten
        '''
        current_season_dataframe.at[team, 'avg_up_to_week_2_playoffs'] = df_current_season.loc[team,'avg_up_to_week_18']
        current_season_dataframe.at[team, 'first_downs_up_to_week_2_playoffs'] = df_current_season.loc[team,'first_downs_up_to_week_18']
        current_season_dataframe.at[team, 'wins_up_to_week_2_playoffs'] = 0.0
        current_season_dataframe.at[team, 'third_down_percentage_up_to_week_2_playoffs'] = df_current_season.loc[team,'third_down_percentage_up_to_week_18'] 
        current_season_dataframe.at[team, 'time_of_possession_up_to_week_2_playoffs'] = df_current_season.loc[team,'time_of_possession_up_to_week_18']


    
    # loop through each post seasons weeks games
    if season != 2020:
        for week in range(1, 5):
            df_week_data = pd.read_csv('./data/postseason_game_data/{0}/week_{1}_data.csv'.format(season, week))
            
            for _, row in df_week_data.iterrows():
                # ---------------------------------------update each teams score per week------------------------------------------
                current_season_dataframe.loc[row.Team,row.Week] = row.Score
                current_season_dataframe[row.Week] = current_season_dataframe[row.Week].fillna(0.0)

                # ---------------------------------------update each teams games played -------------------------------------------
                current_season_dataframe.loc[row.Team,'GamesPlayed'] += 1

                # ----------------------------------------------update score to week------------------------------------------------
                current_season_dataframe.loc[row.Team,'TotalScore'] += row.Score 

                update_avg_score = current_season_dataframe.loc[row.Team,'TotalScore'] / current_season_dataframe.loc[row.Team,'GamesPlayed']

                current_season_dataframe.loc[row.Team,'avg_up_to_week_{0}_playoffs'.format(row.Week + 1)] = update_avg_score 

                if row.Week > 1:
                    current_season_dataframe['avg_up_to_week_{0}_playoffs'.format(row.Week + 1)] = current_season_dataframe['avg_up_to_week_{0}_playoffs'.format(row.Week + 1)].fillna(0.0)
                else:
                    current_season_dataframe['avg_up_to_week_{0}_playoffs'.format(row.Week + 1)] = current_season_dataframe['avg_up_to_week_{0}_playoffs'.format(row.Week + 1)].fillna(current_season_dataframe.loc[row.Team,'avg_up_to_week_{0}_playoffs'.format(row.Week)])

                # ----------------------------------- store wins per team up to a certain week ----------------------------------
                '''
                after week 1 we can make wins up to week in playoffs zero for every team because they will all have the same
                amount of wins at any given week otherwise they wouldn't be there
                '''
                current_season_dataframe.loc[row.Team,'wins_up_to_week_{0}_playoffs'.format(row.Week + 1)] = 0.0
                current_season_dataframe = current_season_dataframe.fillna(0.0)

                # ----------------------------------- store first downs per team up to certain week ----------------------------------
                current_season_dataframe.loc[row.Team,'TotalFirstDowns'] += row.FirstDowns 

                update_first_downs = current_season_dataframe.loc[row.Team,'TotalFirstDowns'] / current_season_dataframe.loc[row.Team,'GamesPlayed']
                current_season_dataframe.loc[row.Team,'first_downs_up_to_week_{0}_playoffs'.format(row.Week + 1)] = update_first_downs
                
                if row.Week > 1:
                    current_season_dataframe['first_downs_up_to_week_{0}_playoffs'.format(row.Week + 1)] = current_season_dataframe['first_downs_up_to_week_{0}_playoffs'.format(row.Week + 1)].fillna(0)
                else:
                    current_season_dataframe['first_downs_up_to_week_{0}_playoffs'.format(row.Week + 1)] = current_season_dataframe['first_downs_up_to_week_{0}_playoffs'.format(row.Week + 1)].fillna(current_season_dataframe.loc[row.Team,'first_downs_up_to_week_{0}_playoffs'.format(row.Week)])

                # ----------------------------------- calculate third down % per team up to certain week ----------------------------------
                current_season_dataframe.loc[row.Team,'ThirdDownPercentage'] += row.ThirdDownPercentage 

                update_third_down_percentage = current_season_dataframe.loc[row.Team,'ThirdDownPercentage'] / current_season_dataframe.loc[row.Team,'GamesPlayed']

                current_season_dataframe.loc[row.Team,'third_down_percentage_up_to_week_{0}_playoffs'.format(row.Week + 1)] = update_third_down_percentage

                if row.Week > 1:
                    current_season_dataframe['third_down_percentage_up_to_week_{0}_playoffs'.format(row.Week + 1)] = current_season_dataframe['third_down_percentage_up_to_week_{0}_playoffs'.format(row.Week + 1)].fillna(0)
                else:
                    current_season_dataframe['third_down_percentage_up_to_week_{0}_playoffs'.format(row.Week + 1)] = current_season_dataframe['third_down_percentage_up_to_week_{0}_playoffs'.format(row.Week + 1)].fillna(current_season_dataframe.loc[row.Team,'third_down_percentage_up_to_week_{0}_playoffs'.format(row.Week)])

                # ----------------------------------- calculate average time of possession per team ----------------------------------
                team_top = float('{0}.{1}'.format(row.TimeOfPossessionMinutes,row.TimeOfPossessionSeconds))

                current_season_dataframe.loc[row.Team,'TotalTimeOfPossession'] += team_top

                update_time_of_possession = current_season_dataframe.loc[row.Team,'TotalTimeOfPossession'] / current_season_dataframe.loc[row.Team,'GamesPlayed']

                current_season_dataframe.loc[row.Team,'time_of_possession_up_to_week_{0}_playoffs'.format(row.Week + 1)] = update_time_of_possession
                
                if row.Week > 1:
                    current_season_dataframe['time_of_possession_up_to_week_{0}_playoffs'.format(row.Week + 1)] = current_season_dataframe['time_of_possession_up_to_week_{0}_playoffs'.format(row.Week + 1)].fillna(0)
                else:
                    current_season_dataframe['time_of_possession_up_to_week_{0}_playoffs'.format(row.Week + 1)] = current_season_dataframe['time_of_possession_up_to_week_{0}_playoffs'.format(row.Week + 1)].fillna(current_season_dataframe.loc[row.Team,'time_of_possession_up_to_week_{0}_playoffs'.format(row.Week)])
        return current_season_dataframe

In [241]:
# function for custom matchup dataframes
def custom_season_games_dataframes(season):
    # load that whole seasons data
    df_regular_season = pd.read_csv('./data/scores_by_regular_season/{0}_data.csv'.format(str(season)))
    df_custom_regular_season = pd.read_csv('./data/custom_season_team_data/{0}_data.csv'.format(str(season)))
    df_custom_regular_season = df_custom_regular_season.set_index('Team')
   
    if season == 2017:
        df_regular_season = df_regular_season[df_regular_season.Week > 1]
        
    if season == 2020:
        # haven't played all 17 weeks yet
        df_regular_season = df_regular_season[df_regular_season.Week < 12]
        
        # get rid of covid cancelled games
        df_regular_season = df_regular_season.dropna(how='any',axis=0) 
        
    # get desired columns   
    df_regular_season = df_regular_season[["HomeTeam", "AwayTeam", "HomeScore", "AwayScore", "Week"]]
    
    # Engineered two columns
    df_regular_season["HomeResult"] = np.where(df_regular_season["HomeScore"] > df_regular_season["AwayScore"], 1, 0)
    df_regular_season["AwayResult"] = np.where(df_regular_season["AwayScore"] > df_regular_season["HomeScore"], 1, 0)

    # add required data for each matchup
    for index, row in df_regular_season.iterrows():
        df_regular_season.at[index, 'AwayAverage'] = df_custom_regular_season.loc[row.AwayTeam,'avg_up_to_week_{0}'.format(row.Week)] 
        df_regular_season.at[index, 'HomeAverage'] = df_custom_regular_season.loc[row.HomeTeam,'avg_up_to_week_{0}'.format(row.Week)] 

        df_regular_season.at[index, 'AwayWins'] = df_custom_regular_season.loc[row.AwayTeam,'wins_up_to_week_{0}'.format(row.Week)] 
        df_regular_season.at[index, 'HomeWins'] = df_custom_regular_season.loc[row.HomeTeam,'wins_up_to_week_{0}'.format(row.Week)] 

        df_regular_season.at[index, 'AwayFirstDowns'] = df_custom_regular_season.loc[row.AwayTeam,'first_downs_up_to_week_{0}'.format(row.Week)] 
        df_regular_season.at[index, 'HomeFirstDowns'] = df_custom_regular_season.loc[row.HomeTeam,'first_downs_up_to_week_{0}'.format(row.Week)] 

        df_regular_season.at[index, 'AwayTime'] = df_custom_regular_season.loc[row.AwayTeam,'time_of_possession_up_to_week_{0}'.format(row.Week)] 
        df_regular_season.at[index, 'HomeTime'] = df_custom_regular_season.loc[row.HomeTeam,'time_of_possession_up_to_week_{0}'.format(row.Week)] 

        df_regular_season.at[index, 'AwayThirdDowns'] = df_custom_regular_season.loc[row.AwayTeam,'third_down_percentage_up_to_week_{0}'.format(row.Week)] 
        df_regular_season.at[index, 'HomeThirdDowns'] = df_custom_regular_season.loc[row.HomeTeam,'third_down_percentage_up_to_week_{0}'.format(row.Week)] 
    
    # remove week column so we can also add playoff games to the end
    df_regular_season = df_regular_season.drop(columns=['Week'])   
    
    
    # in case post season doesn't exist (2020)
    if season != 2020:
        # load that whole post season data
        df_post_season = pd.read_csv('./data/scores_by_post_season/{0}_data.csv'.format(str(season)))
        df_custom_post_season = pd.read_csv('./data/custom_post_season_team_data/{0}_data.csv'.format(str(season)))
        
        df_custom_post_season = df_custom_post_season.set_index('Team')
        
        # get desired columns
        df_post_season = df_post_season[["HomeTeam", "AwayTeam", "HomeScore", "AwayScore", "Week"]]
        
        # Engineered two columns
        df_post_season["HomeResult"] = np.where(df_post_season["HomeScore"] > df_post_season["AwayScore"], 1, 0)
        df_post_season["AwayResult"] = np.where(df_post_season["AwayScore"] > df_post_season["HomeScore"], 1, 0)

        
        for index, row in df_post_season.iterrows():
            df_post_season.at[index, 'AwayAverage'] = df_custom_post_season.loc[row.AwayTeam,'avg_up_to_week_{0}_playoffs'.format(row.Week)] 
            df_post_season.at[index, 'HomeAverage'] = df_custom_post_season.loc[row.HomeTeam,'avg_up_to_week_{0}_playoffs'.format(row.Week)] 

            df_post_season.at[index, 'AwayWins'] = df_custom_post_season.loc[row.AwayTeam,'wins_up_to_week_{0}_playoffs'.format(row.Week)] 
            df_post_season.at[index, 'HomeWins'] = df_custom_post_season.loc[row.HomeTeam,'wins_up_to_week_{0}_playoffs'.format(row.Week)] 

            df_post_season.at[index, 'AwayFirstDowns'] = df_custom_post_season.loc[row.AwayTeam,'first_downs_up_to_week_{0}_playoffs'.format(row.Week)] 
            df_post_season.at[index, 'HomeFirstDowns'] = df_custom_post_season.loc[row.HomeTeam,'first_downs_up_to_week_{0}_playoffs'.format(row.Week)] 

            df_post_season.at[index, 'AwayTime'] = df_custom_post_season.loc[row.AwayTeam,'time_of_possession_up_to_week_{0}_playoffs'.format(row.Week)] 
            df_post_season.at[index, 'HomeTime'] = df_custom_post_season.loc[row.HomeTeam,'time_of_possession_up_to_week_{0}_playoffs'.format(row.Week)] 

            df_post_season.at[index, 'AwayThirdDowns'] = df_custom_post_season.loc[row.AwayTeam,'third_down_percentage_up_to_week_{0}_playoffs'.format(row.Week)] 
            df_post_season.at[index, 'HomeThirdDowns'] = df_custom_post_season.loc[row.HomeTeam,'third_down_percentage_up_to_week_{0}_playoffs'.format(row.Week)] 
    
        # remove week column so we can also add playoff games to the end
        df_post_season = df_post_season.drop(columns=['Week'])  
        
         # combine 2 dataframes
        df_regular_season = df_regular_season.append(df_post_season, ignore_index = True) 
                                                                                                  
                                                                                                  
   
    return df_regular_season
    

In [288]:
# create list of seasons
seasons = [2017, 2018,2019,2020]


for season in seasons:
    # put regular season stats in csv
    df = custom_regular_season_dataframe(season)
    df.to_csv('./data/custom_season_team_data/{0}_data.csv'.format(str(season)), header=True,  encoding='utf-8') 

    # put post season stats in csv
    if season != 2020:
        df = custom_regular_season_dataframe(season)
        df.to_csv('./data/custom_post_season_team_data/{0}_data.csv'.format(str(season)), header=True,  encoding='utf-8') 

    # use all games and results for 
    df = custom_regular_season_dataframe(season)
    df.to_csv('./data/custom_games_by_season/{0}_data.csv'.format(str(season)), header=True,  encoding='utf-8') 

# for season in seasons:
#     custom_dataframes[season] = custom_regular_season_dataframe(season)

In [280]:
custom_dataframes[2019]

,TotalScore,TotalWins,TotalFirstDowns,ThirdDownPercentage,TotalTimeOfPossession,GamesPlayed,avg_up_to_week_1,first_downs_up_to_week_1,wins_up_to_week_1,third_down_percentage_up_to_week_1,time_of_possession_up_to_week_1,1,avg_up_to_week_2,wins_up_to_week_2,first_downs_up_to_week_2,third_down_percentage_up_to_week_2,time_of_possession_up_to_week_2,2,avg_up_to_week_3,wins_up_to_week_3,first_downs_up_to_week_3,third_down_percentage_up_to_week_3,time_of_possession_up_to_week_3,3,avg_up_to_week_4,wins_up_to_week_4,first_downs_up_to_week_4,third_down_percentage_up_to_week_4,time_of_possession_up_to_week_4,4,avg_up_to_week_5,wins_up_to_week_5,first_downs_up_to_week_5,third_down_percentage_up_to_week_5,time_of_possession_up_to_week_5,5,avg_up_to_week_6,wins_up_to_week_6,first_downs_up_to_week_6,third_down_percentage_up_to_week_6,time_of_possession_up_to_week_6,6,avg_up_to_week_7,wins_up_to_week_7,first_downs_up_to_week_7,third_down_percentage_up_to_week_7,time_of_possession_up_to_week_7,7,avg_up_to_week_8,wins_up_to_week_8,first_downs_up_to_week_8,third_down_percentage_up_to_week_8,time_of_possession_up_to_week_8,8,avg_up_to_week_9,wins_up_to_week_9,first_downs_up_to_week_9,third_down_percentage_up_to_week_9,time_of_possession_up_to_week_9,9,avg_up_to_week_10,wins_up_to_week_10,first_downs_up_to_week_10,third_down_percentage_up_to_week_10,time_of_possession_up_to_week_10,10,avg_up_to_week_11,wins_up_to_week_11,first_downs_up_to_week_11,third_down_percentage_up_to_week_11,time_of_possession_up_to_week_11,11,avg_up_to_week_12,wins_up_to_week_12,first_downs_up_to_week_12,third_down_percentage_up_to_week_12,time_of_possession_up_to_week_12,12,avg_up_to_week_13,wins_up_to_week_13,first_downs_up_to_week_13,third_down_percentage_up_to_week_13,time_of_possession_up_to_week_13,13,avg_up_to_week_14,wins_up_to_week_14,first_downs_up_to_week_14,third_down_percentage_up_to_week_14,time_of_possession_up_to_week_14,14,avg_up_to_week_15,wins_up_to_week_15,first_downs_up_to_week_15,third_down_percentage_up_to_week_15,time_of_possession_up_to_week_15,15,avg_up_to_week_16,wins_up_to_week_16,first_downs_up_to_week_16,third_down_percentage_up_to_week_16,time_of_possession_up_to_week_16,16,avg_up_to_week_17,wins_up_to_week_17,first_downs_up_to_week_17,third_down_percentage_up_to_week_17,time_of_possession_up_to_week_17,17,avg_up_to_week_18,wins_up_to_week_18,first_downs_up_to_week_18,third_down_percentage_up_to_week_18,time_of_possession_up_to_week_18
Team,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ARI,361,5,314,558.8,441.27,16,14.0625,14.9375,0.0,29.1,27.33,27.0,27.0,0.0,21.0,40.9,30.37,17.0,22.0,0.0,18.0,29.55,26.295,20.0,21.333333,0.0,19.000000,37.333333,28.010000,10.0,18.50,0.0,19.25,36.325,27.5975,26.0,20.00,1.0,20.60,38.300,28.3380,34.0,22.333333,2.0,21.166667,40.883333,28.136667,27.0,23.000000,3.0,20.714286,41.714286,28.344286,9.0,21.250000,3.0,19.375000,38.587500,27.563750,25.0,21.666667,3.0,19.333333,37.077778,27.311111,27.0,22.200000,3.0,19.400000,34.370000,26.937000,26.0,22.545455,3.0,19.545455,33.972727,27.327273,0.0,22.545455,0.0,19.545455,33.972727,27.327273,7.0,21.250000,3.0,19.000000,32.425000,27.152500,17.0,20.923077,3.0,18.846154,32.730769,27.263846,38.0,22.142857,4.0,19.357143,33.964286,27.395714,27.0,22.466667,5.0,19.466667,33.920000,27.871333,24.0,22.5625,5.0,19.6250,34.92500,27.579375
ATL,381,7,383,673.3,492.90,16,25.8750,22.0000,0.0,45.3,30.36,12.0,12.0,0.0,27.0,25.0,30.13,24.0,18.0,1.0,23.0,27.50,28.865,24.0,20.000000,1.0,23.666667,45.000000,28.700000,10.0,17.50,1.0,23.75,42.675,29.1275,32.0,20.40,1.0,23.40,45.380,29.1020,33.0,22.500000,1.0,24.666667,46.900000,29.663333,10.0,20.714286,1.0,23.142857,43.771429,28.641429,20.0,20.625000,1.0,24.000000,42.462500,28.870000,0.0,18.333333,0.0,21.333333,37.744444,25.662222,26.0,21.222222,2.0,24.000000,42.188889,29.380000,29.0,22.000000,3.0,23.600000,42.260000,29.568000,22.0,22.000000,3.0,23.454545,40.690909,29.732727,18.0,21